In [2]:
import pymysql
import pandas as pd

try:
    # Conexión a la base de datos del simulador
    connection = pymysql.connect(
        host='212.227.90.6',
        user='Equipo12',
        password='E1q2u3i4p5o12',
        database='Equip_12',
        port=3306
    )
    print("✅ Conexión establecida con MySQL Server")

    # Crear un cursor
    cursor = connection.cursor()

    # Consultar las tablas disponibles
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()

    # Diccionario para almacenar los dataframes
    dataframes = {}

    for tabla in tables:
        table_name = tabla[0]
        print(f"📥 Cargando tabla: {table_name}")
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql(query, connection)
        dataframes[table_name] = df
        print(f"✅ DataFrame para '{table_name}' creado con {df.shape[0]} filas")

except pymysql.MySQLError as e:
    print("❌ Error al conectar con MySQL:", e)


✅ Conexión establecida con MySQL Server
📥 Cargando tabla: RRHH


/var/folders/zv/gcn00wl120d9rg594_6wxmvr0000gn/T/ipykernel_95053/2277306723.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


✅ DataFrame para 'RRHH' creado con 740 filas


In [3]:
df_rrhh = dataframes['RRHH']

In [4]:
df_rrhh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       740 non-null    int64 
 1   Reason_absence           740 non-null    int64 
 2   Month_absence            740 non-null    int64 
 3   Day_week                 740 non-null    int64 
 4   Seasons                  740 non-null    int64 
 5   Transportation_expense   740 non-null    int64 
 6   Distance_Residence_Work  740 non-null    int64 
 7   Service_time             740 non-null    int64 
 8   Age                      740 non-null    int64 
 9   Work_load_Average_day    740 non-null    object
 10  Hit_target               740 non-null    int64 
 11  Disciplinary_failure     740 non-null    object
 12  Education                740 non-null    object
 13  Son                      740 non-null    object
 14  Social_drinker           740 non-null    o

1- Tratamiento de valores faltantes

In [6]:
df_rrhh.isna().sum()

# No hay nulos

ID                         0
Reason_absence             0
Month_absence              0
Day_week                   0
Seasons                    0
Transportation_expense     0
Distance_Residence_Work    0
Service_time               0
Age                        0
Work_load_Average_day      0
Hit_target                 0
Disciplinary_failure       0
Education                  0
Son                        0
Social_drinker             0
Social_smoker              0
Pet                        0
Weight                     0
Height                     0
Body_mass_index            0
Absenteeism_hours          0
dtype: int64

2- Eliminación o corrección de duplicados

In [20]:
# Duplicados completos
df_rrhh.duplicated().sum()

np.int64(34)

In [7]:
# Ver los duplicados concretos
df_rrhh[df_rrhh.duplicated()]

# Se han detectado 34 filas completamente duplicadas, algunas repetidas hasta 7 veces, como en el caso del ID 3 con el mismo motivo, mes y carga horaria.
# Aunque un mismo empleado puede tener varias ausencias similares, la duplicación exacta de todos los campos en múltiples filas consecutivas sugiere errores de carga de datos.
# Recomiendo dejarlo marcado como “candidato a eliminación” y decidirlo con el equipo antes de aplicar drop_duplicates().


,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,...,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
92,10,22,12,4,4,361,52,3,28,"261,306",...,0,1,1,1,0,4,80,172,27,8
348,34,23,10,3,4,118,10,10,37,"253,465",...,0,1,0,0,0,0,83,172,28,3
383,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
386,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
387,3,27,2,6,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
388,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
389,3,27,2,6,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
396,22,23,5,4,3,179,26,9,30,"246,074",...,0,3,0,0,0,0,56,171,19,3
412,28,23,12,4,4,225,26,9,28,"280,549",...,0,1,1,0,0,2,69,169,24,3
431,3,27,3,5,2,179,51,18,38,"222,196",...,0,1,0,1,0,0,89,170,31,3


3- Validación de valores atípicos (outliers)

In [8]:
df_rrhh.describe()

# Se han encontrado posibles valores extremos en:
# Transportation_expense (máx. 388), la media es 221
# Absenteeism_hours (máx. 120)
# Work_load_Average_day (máx. 310), muy cercano al máximo permitido en carga diaria

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Hit_target,Weight,Height,Body_mass_index,Absenteeism_hours
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,18.017568,19.216216,6.324324,3.914865,2.544595,221.329730,29.631081,12.554054,36.450000,94.587838,79.035135,172.114865,26.677027,6.924324
std,11.021247,8.433406,3.436287,1.421675,1.111831,66.952223,14.836788,4.384873,6.478772,3.779313,12.883211,6.034995,4.285452,13.330998
min,1.000000,0.000000,0.000000,2.000000,1.000000,118.000000,5.000000,1.000000,27.000000,81.000000,56.000000,163.000000,19.000000,0.000000
25%,9.000000,13.000000,3.000000,3.000000,2.000000,179.000000,16.000000,9.000000,31.000000,93.000000,69.000000,169.000000,24.000000,2.000000
50%,18.000000,23.000000,6.000000,4.000000,3.000000,225.000000,26.000000,13.000000,37.000000,95.000000,83.000000,170.000000,25.000000,3.000000
75%,28.000000,26.000000,9.000000,5.000000,4.000000,260.000000,50.000000,16.000000,40.000000,97.000000,89.000000,172.000000,31.000000,8.000000
max,36.000000,28.000000,12.000000,6.000000,4.000000,388.000000,52.000000,29.000000,58.000000,100.000000,108.000000,196.000000,38.000000,120.000000


4- Estandarización de formatos

In [11]:
df_rrhh['Month_absence'].value_counts()

# En la columna Month_absence se ha detectado el valor 0, que no corresponde a ningún mes válido (1–12). Solo hay 3 registros con este valor. Se sugiere marcarlo como anómalo y decidir si imputarlo (por moda o mediana), dejarlo como está o excluirlo del análisis final.


Month_absence
3     87
2     72
10    71
7     67
5     64
11    63
6     54
8     54
4     53
9     53
1     50
12    49
0      3
Name: count, dtype: int64

In [15]:
df_rrhh['Disciplinary_failure'].value_counts()

Disciplinary_failure
0    700
1     40
Name: count, dtype: int64

In [16]:
df_rrhh['Education'].value_counts()

Education
1    611
3     79
2     46
4      4
Name: count, dtype: int64

In [17]:
df_rrhh['Social_drinker'].value_counts()

Social_drinker
1    420
0    320
Name: count, dtype: int64

In [18]:
df_rrhh['Social_smoker'].value_counts()

Social_smoker
0    686
1     54
Name: count, dtype: int64

5- Corrección de tipos de datos

In [19]:
df_rrhh.dtypes
#En la revisión de tipos de datos se ha detectado que 7 columnas aparecen como object cuando no deberían:
# Work_load_Average_day debería ser float, pero contiene comas como separadores
# Disciplinary_failure, Social_drinker, Social_smoker, Education, Son y Pet deberían ser int o category

ID                          int64
Reason_absence              int64
Month_absence               int64
Day_week                    int64
Seasons                     int64
Transportation_expense      int64
Distance_Residence_Work     int64
Service_time                int64
Age                         int64
Work_load_Average_day      object
Hit_target                  int64
Disciplinary_failure       object
Education                  object
Son                        object
Social_drinker             object
Social_smoker              object
Pet                        object
Weight                      int64
Height                      int64
Body_mass_index             int64
Absenteeism_hours           int64
dtype: object

In [22]:
pd.set_option('display.max_columns', None) # para ver todas las columnas y que no se corten por el limite de Panda


In [23]:
df_rrhh.sample(40)

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
350,20,28,10,6,4,260,50,11,36,"253,465",93,0,1,4,1,0,0,65,168,23,3
565,28,7,2,2,2,225,26,9,28,"264,249",97,0,1,1,0,0,2,69,169,24,2
75,2,18,8,5,1,235,29,12,48,"205,917",92,0,1,1,0,1,5,88,163,33,8
78,2,28,8,6,1,235,29,12,48,"205,917",92,0,1,1,0,1,5,88,163,33,8
456,3,23,8,6,1,179,51,18,38,"205,917",92,0,1,0,1,0,0,89,170,31,2
136,18,1,7,2,1,330,16,4,28,"275,312",98,0,2,0,0,0,0,84,182,25,8
205,5,26,7,2,1,235,20,13,43,"230,290",92,0,1,1,1,0,0,106,167,38,8
112,11,6,3,5,2,289,36,13,33,"343,253",95,0,1,2,1,0,1,90,172,30,8
162,24,13,11,6,4,246,25,16,41,"284,031",97,0,1,0,1,0,0,67,170,23,8
420,3,27,2,4,2,179,51,18,38,"264,249",97,0,1,0,1,0,0,89,170,31,3
